In [1]:
import os, sys

# Ustal ścieżkę do katalogu głównego projektu (tam, gdzie masz folder src/)
project_root = os.path.abspath(os.path.join(".."))
if project_root not in sys.path:
    sys.path.append(project_root)

print("✅ Dodano ścieżkę projektu:", project_root)


✅ Dodano ścieżkę projektu: C:\GitHubRepo\data-science\projects\ETH-volitality-fotecasting


In [ ]:
import sys, os
sys.path.append(os.path.abspath("../src"))

from evaluation import evaluate_egarch

# Rolling refit forecast (adaptacyjny)
evaluate_egarch(n_obs=1100)




📂 Wczytuję model z egarch_ETH_5m_p1_o1_q1_t.pkl...
📈 Wczytuję dane testowe...
✅ Dane załadowane poprawnie (train + test).
Train shape: (17279, 11), Test shape: (8927, 11)
Train range: 2024-01-01 00:05:00 → 2024-02-29 23:55:00
Test range:  2024-03-01 00:05:00 → 2024-03-31 23:55:00
📏 Używam tylko pierwszych 1100 świeczek z testu.
🔮 Liczę rolling forecast wariancji...
